### Linear Regression

In [20]:
import pandas as pd
import os
import sys
import numpy as np
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(parent_dir)

from cross_validation import cross_validate
from metrics import results_to_mean_confidence_interval
from transformations import get_transforms

In [21]:

# M repetitions of 3-fold cross-validated linear regression
M_ = {
   "RSIVL": 20,
   "VISC": 2,
   "Sav. Scenes": 2,
   "Sav. Objects": 2,
   "IC9. Scenes": 1,
   "IC9. Objects": 1,
   "IC9. Person": 1,
   "IC9. Transport": 1,
   "IC9. Architecture": 1,
   "IC9. Abstract": 1,
   "IC9. Advertisement": 1,
   "Sav. Art": 2,
   "Sav. Suprematism": 2,
   "IC9. Paintings": 1,
   "Sav. Interior Design": 2,
   "SVG": 20,
}

# Define dataset combinations
feature_map = {
   "RSIVL": "../features/RSIVL.csv",
   "VISC": "../features/VISC.csv",
   "Sav. Scenes": "../features/SAVOIAS Scenes.csv",
   "Sav. Objects": "../features/SAVOIAS Objects.csv",
   "IC9. Scenes": "../features/IC9600 Scenes.csv",
   "IC9. Objects": "../features/IC9600 Objects.csv",
   "IC9. Person": "../features/IC9600 Person.csv",
   "IC9. Transport": "../features/IC9600 Transport.csv",
   "IC9. Architecture": "../features/IC9600 Architecture.csv",
   "Sav. Art": "../features/SAVOIAS Art.csv",
   "Sav. Suprematism": "../features/SAVOIAS Suprematism.csv",
   "IC9. Paintings": "../features/IC9600 Paintings.csv",
   "Sav. Interior Design": "../features/SAVOIAS Interior Design.csv",
   "IC9. Abstract": "../features/IC9600 Abstract.csv",
   "IC9. Advertisement": "../features/IC9600 Advertisement.csv",
   "SVG": "../features/SVG.csv"
}

In [ ]:
for dataset in feature_map:
    
    features = pd.read_csv(feature_map[dataset])
    features = features.rename(columns={"# of SAM segmentations": "num_seg_64points", "# of FC-CLIP classes": "num_classes", "gemini surprise score": "gemini_surprise_score", "normalized_rating": "complexity"})
    
    model_strs = [
    "sqrt_seg_64points + sqrt_num_classes",
    "sqrt_seg_64points + sqrt_num_classes + MSG",
    "sqrt_seg_64points + sqrt_num_classes + MUC3",
    "sqrt_seg_64points + sqrt_num_classes + MUC4",
    "sqrt_seg_64points + sqrt_num_classes + MUC5",
    "sqrt_seg_64points + sqrt_num_classes + MUC6",
    "sqrt_seg_64points + sqrt_num_classes + MUC7",
    "sqrt_seg_64points + sqrt_num_classes + MUC8",
    "sqrt_seg_64points + sqrt_num_classes + MUC3 + MSG",
    "sqrt_seg_64points + sqrt_num_classes + MUC4 + MSG",
    "sqrt_seg_64points + sqrt_num_classes + MUC5 + MSG",
    "sqrt_seg_64points + sqrt_num_classes + MUC6 + MSG",
    "sqrt_seg_64points + sqrt_num_classes + MUC7 + MSG",
    "sqrt_seg_64points + sqrt_num_classes + MUC8 + MSG",
    "MUC3 + MSG",
    "MUC4 + MSG",
    "MUC5 + MSG",
    "MUC6 + MSG",
    "MUC7 + MSG",
    "MUC8 + MSG",
    "sqrt_seg_64points + sqrt_num_classes + MUC3 + MSG + gemini_surprise_score",
    "sqrt_seg_64points + sqrt_num_classes + MUC4 + MSG + gemini_surprise_score",
    "sqrt_seg_64points + sqrt_num_classes + MUC5 + MSG + gemini_surprise_score",
    "sqrt_seg_64points + sqrt_num_classes + MUC6 + MSG + gemini_surprise_score",
    "sqrt_seg_64points + sqrt_num_classes + MUC7 + MSG + gemini_surprise_score",
    "sqrt_seg_64points + sqrt_num_classes + MUC8 + MSG + gemini_surprise_score",
    "sqrt_seg_64points + sqrt_num_classes + gemini_surprise_score",
    "IC9600"
    ]
    
    # remove any NaN

    len_before = len(features)
    features = features.dropna(subset=['gemini_surprise_score'])
    len_after = len(features)
    print("Removed {} NaNs (gemini surprise score) from dataset {}".format(len_before - len_after, dataset))
    
    dataset_result = {}
    def run_regression(data, N, M, target, model_strs, return_preds):
        df = data.copy()
        get_transforms(df)
        return cross_validate(df, N=N, M=M, target=target, model_strs=model_strs, return_preds=return_preds)

    dataset_result[dataset] = run_regression(features, N=3, M=M_[dataset], target="complexity", model_strs=model_strs, return_preds=False) 
    # Remove predictions from dataset_result
    for dataset in dataset_result:
        for model in dataset_result[dataset]:
            if 'predictions' in dataset_result[dataset][model]:
                del dataset_result[dataset][model]['predictions']
    results_stats = results_to_mean_confidence_interval(dataset_result)
    print("Reporting Spearman correlations.")

    # sort according to correlation first
    mod_to_corr = {}

    for dset, v in results_stats.items():
        print("\nDATASET: {}".format(dataset))
        for mod, vv in v.items():
            for s, (m, i) in vv.items():
                if s == "spearman_test":
                    mod_to_corr[mod] = m
                    
    for mod, corr in sorted(mod_to_corr.items(), key=lambda x: x[1], reverse=True):
        print("{}: {}".format(mod, corr))
    print("<", "-"*40, ">")

Series([], Name: image_id, dtype: object)
Reporting Spearman correlations.

DATASET: RSIVL
sqrt_seg_64points + sqrt_num_classes + MUC4 + MSG + gemini_surprise_score: 0.8405226386639649
sqrt_seg_64points + sqrt_num_classes + MSG: 0.839639324711659
sqrt_seg_64points + sqrt_num_classes + MUC3 + MSG + gemini_surprise_score: 0.8386518108472312
sqrt_seg_64points + sqrt_num_classes + MUC5 + MSG + gemini_surprise_score: 0.8364659389233141
sqrt_seg_64points + sqrt_num_classes + MUC3 + MSG: 0.8364359525762958
sqrt_seg_64points + sqrt_num_classes + MUC4 + MSG: 0.8346794166285834
sqrt_seg_64points + sqrt_num_classes + MUC5 + MSG: 0.8342300171232873
sqrt_seg_64points + sqrt_num_classes + MUC8 + MSG + gemini_surprise_score: 0.8324321363363048
sqrt_seg_64points + sqrt_num_classes + MUC7 + MSG + gemini_surprise_score: 0.8317549858436283
sqrt_seg_64points + sqrt_num_classes + MUC6 + MSG + gemini_surprise_score: 0.8316700920363501
sqrt_seg_64points + sqrt_num_classes + MUC8 + MSG: 0.8316591729830273
sqr

### Correlations

In [7]:
import pandas as pd
import numpy as np
from scipy import stats
import os

def analyze_file_correlations(data, name=None):
    """Analyze Spearman correlations between 'complexity' and other features in a dataset."""
    try:
        # If 'data' is a file path string, read the CSV
        if isinstance(data, str):
            df = pd.read_csv(data)
        else:
            df = data

        if 'complexity' not in df.columns:
            print(f"\nWarning: 'complexity' not found in {name or 'dataset'}")
            return None, 0

        correlations = {}
        for column in df.columns:
            if column not in ['complexity', 'image_id']:
                # Compute Spearman correlation (ignoring NaNs)
                correlation, _ = stats.spearmanr(df['complexity'], df[column], nan_policy='omit')
                if not np.isnan(correlation):
                    correlations[column] = correlation

        return correlations, len(df)

    except Exception as e:
        print(f"\nError processing {name or 'dataset'}: {e}")
        return None, 0

def aggregate_correlations(all_files):
    """Aggregate correlations across all individual CSV files."""
    all_correlations = {}  # Dictionary to collect correlations per feature
    all_datasets = []      # List to store (DataFrame, dataset_name) tuples

    # Process each individual file
    for file in all_files:
        try:
            df = pd.read_csv(file)
            dataset_name = os.path.basename(file)
            all_datasets.append((df, dataset_name))
        except Exception as e:
            print(f"Error reading {file}: {e}")

    # Iterate over every dataset
    for data, name in all_datasets:
        correlations, n_samples = analyze_file_correlations(data, name)
        if correlations:
            for feature, correlation in correlations.items():
                # Append a tuple of (correlation, sample size, dataset name) for each feature
                all_correlations.setdefault(feature, []).append((correlation, n_samples, name))

    # Compute statistics for each feature across datasets
    feature_stats = {}
    total_datasets = len(all_datasets)
    for feature, corr_list in all_correlations.items():
        coverage = len(corr_list) / total_datasets
        # Extract correlation values and sample sizes
        corr_values = [item[0] for item in corr_list]
        sample_sizes = [item[1] for item in corr_list]
        total_samples = sum(sample_sizes)

        # Unweighted (simple) average and standard deviation over datasets
        mean_corr = np.mean(corr_values)
        std_corr = np.std(corr_values) if len(corr_values) > 1 else 0

        # Weighted average and weighted standard deviation
        weights = [n / total_samples for n in sample_sizes]
        weighted_mean = sum(c * w for c, w in zip(corr_values, weights))
        weighted_var = sum(w * (x - weighted_mean) ** 2 for x, w in zip(corr_values, weights))
        weighted_std = np.sqrt(weighted_var)

        feature_stats[feature] = {
            'mean_correlation': mean_corr,
            'std_correlation': std_corr,
            'weighted_mean_correlation': weighted_mean,
            'weighted_std_correlation': weighted_std,
            'num_datasets': len(corr_list),
            'dataset_coverage': coverage,
            'total_samples': total_samples
        }

    # Convert the dictionary to a DataFrame and add columns for absolute values for sorting
    stats_df = pd.DataFrame.from_dict(feature_stats, orient='index')
    stats_df['abs_mean_correlation'] = stats_df['mean_correlation'].abs()
    stats_df['abs_weighted_mean_correlation'] = stats_df['weighted_mean_correlation'].abs()

    return stats_df

def print_unweighted_features_table(stats_df):
    """Print table of features using unweighted (simple average) statistics."""
    # Sort by absolute mean correlation (largest first)
    sorted_features = stats_df.sort_values('abs_mean_correlation', ascending=False)
    print("\nUnweighted Averages Over Datasets:")
    print("-" * 70)
    print(f"{'Feature':<30} | {'Mean Corr':>9} | {'Std Dev':>8} | {'Datasets':>8}")
    print("-" * 70)
    for feature, row in sorted_features.iterrows():
        print(f"{feature[:30]:<30} | {row['mean_correlation']:9.3f} | {row['std_correlation']:8.3f} | {row['num_datasets']:8}")

def print_weighted_features_table(stats_df):
    """Print table of features using weighted statistics based on dataset sample sizes."""
    # Sort by absolute weighted mean correlation (largest first)
    sorted_features = stats_df.sort_values('abs_weighted_mean_correlation', ascending=False)
    print("\nWeighted Averages Based on Dataset Sample Sizes:")
    print("-" * 90)
    print(f"{'Feature':<30} | {'Weighted Mean':>15} | {'Weighted Std':>12} | {'Total Samples':>15}")
    print("-" * 90)
    for feature, row in sorted_features.iterrows():
        print(f"{feature[:30]:<30} | {row['weighted_mean_correlation']:15.3f} | {row['weighted_std_correlation']:12.3f} | {row['total_samples']:15,}")

# ----------------------------
# Example Usage
# ----------------------------

# List of individual CSV files containing features
feature_files = [
    "RSIVL.csv",
    "VISC.csv",
    "IC9600 Abstract.csv",
    "IC9600 Paintings.csv",
    "IC9600 Scenes.csv",
    "SAVOIAS Objects.csv",
    "SAVOIAS Art.csv",
    "SAVOIAS Scenes.csv",
    "SAVOIAS Suprematism.csv",
    "SAVOIAS Interior Design.csv",
    "IC9600 Advertisement.csv",
    "IC9600 Architecture.csv",
    "IC9600 Person.csv",
    "IC9600 Transport.csv",
    "IC9600 Objects.csv",
    "SVG.csv"
]

# If your files are in a different folder, set the folder path here
features_folder = "../features"
feature_files = [os.path.join(features_folder, file) for file in feature_files]

# Aggregate correlations over the individual files
stats_df = aggregate_correlations(feature_files)

# Print the two separate tables:
print_unweighted_features_table(stats_df)
print_weighted_features_table(stats_df)



Unweighted Averages Over Datasets:
----------------------------------------------------------------------
Feature                        | Mean Corr |  Std Dev | Datasets
----------------------------------------------------------------------
IC9600                         |     0.877 |    0.057 |     16.0
# of SAM segmentations         |     0.709 |    0.095 |     16.0
MSG                            |     0.583 |    0.091 |     16.0
symmetry                       |    -0.565 |    0.105 |     16.0
M6                             |     0.549 |    0.100 |     16.0
edge density                   |     0.548 |    0.085 |     16.0
clutter                        |     0.545 |    0.111 |     16.0
# of FC-CLIP classes           |     0.534 |    0.182 |     16.0
MUC6                           |     0.530 |    0.148 |     16.0
MUC7                           |     0.529 |    0.156 |     16.0
MUC8                           |     0.525 |    0.161 |     16.0
MUC5                           |     0.522

C:\Users\karab\AppData\Local\Temp\ipykernel_19164\3755367065.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, _ = stats.spearmanr(df['complexity'], df[column], nan_policy='omit')


### Ablation (Permutation Tests)

In [13]:
import pandas as pd
import numpy as np
from scipy import stats
import os

def calculate_correlations(df, dataset_name, feature_x, feature_y, n_permutations=1000):
    """
    Calculate Spearman correlations of `feature_x` and `feature_y` with `complexity`.
    Perform a permutation test to evaluate statistical significance.
    """
    if 'complexity' not in df.columns:
        print(f"Warning: 'complexity' column not found in {dataset_name}")
        return None

    corr_x, _ = stats.spearmanr(df['complexity'], df[feature_x], nan_policy='omit')
    corr_y, _ = stats.spearmanr(df['complexity'], df[feature_y], nan_policy='omit')

    # Permutation test: Compute p-value
    actual_diff = abs(corr_x) - abs(corr_y)
    permuted_diffs = [
        abs(stats.spearmanr(df['complexity'].sample(frac=1).values, df[feature_x], nan_policy='omit')[0]) -
        abs(stats.spearmanr(df['complexity'].sample(frac=1).values, df[feature_y], nan_policy='omit')[0])
        for _ in range(n_permutations)
    ]

    p_value = (np.sum(np.abs(permuted_diffs) >= np.abs(actual_diff)) + 1) / (n_permutations + 1)
    winner = feature_x if abs(corr_x) > abs(corr_y) else feature_y if abs(corr_y) > abs(corr_x) else "Tie"

    return {
        'dataset': dataset_name,
        'correlation_x': corr_x,
        'correlation_y': corr_y,
        'winner': winner,
        'p_value': p_value,
        'n_samples': len(df)
    }

def compare_feature_correlations(file_paths, feature_x, feature_y, base_path="", n_permutations=1000):
    """
    Compare correlations of two features across multiple datasets.
    Prints and returns the correlation results.
    """
    results = []
    print("\n{:<30} | {:>10} | {:>10} | {:<10} | {:<10}".format("Dataset", feature_x, feature_y, "Winner", "p-value"))
    print("-" * 80)

    for file in file_paths:
        try:
            df = pd.read_csv(os.path.join(base_path, file))
            dataset_name = os.path.basename(file).replace('_features.csv', '')
            result = calculate_correlations(df, dataset_name, feature_x, feature_y, n_permutations)
            if result:
                results.append(result)
                print("{:<30} | {:>10.3f} | {:>10.3f} | {:<10} | {:<10.3f}".format(
                    result['dataset'], result['correlation_x'], result['correlation_y'], result['winner'], result['p_value']
                ))
        except Exception as e:
            print(f"Error processing {file}: {e}")

    return results

# ----------------------------
# Example Usage
# ----------------------------

feature_files = [
    "RSIVL.csv",
    "VISC.csv",
    "IC9600 Abstract.csv",
    "IC9600 Paintings.csv",
    "IC9600 Scenes.csv",
    "SAVOIAS Objects.csv",
    "SAVOIAS Art.csv",
    "SAVOIAS Scenes.csv",
    "SAVOIAS Suprematism.csv",
    "SAVOIAS Interior Design.csv",
    "IC9600 Advertisement.csv",
    "IC9600 Architecture.csv",
    "IC9600 Person.csv",
    "IC9600 Transport.csv",
    "IC9600 Objects.csv",
    "SVG.csv"
]

features_folder = "../features"
feature_files = [os.path.join(features_folder, file) for file in feature_files]


# Run correlation comparison
results = compare_feature_correlations(feature_files, 'symmetry', 'MSG')



Dataset                        |   symmetry |        MSG | Winner     | p-value   
--------------------------------------------------------------------------------
RSIVL.csv                      |     -0.708 |      0.721 | MSG        | 0.900     
VISC.csv                       |     -0.579 |      0.573 | symmetry   | 0.837     
IC9600 Abstract.csv            |     -0.663 |      0.705 | MSG        | 0.087     
IC9600 Paintings.csv           |     -0.491 |      0.555 | MSG        | 0.014     
IC9600 Scenes.csv              |     -0.649 |      0.664 | MSG        | 0.506     
SAVOIAS Objects.csv            |     -0.379 |      0.397 | MSG        | 0.741     
SAVOIAS Art.csv                |     -0.394 |      0.474 | MSG        | 0.076     
SAVOIAS Scenes.csv             |     -0.498 |      0.492 | symmetry   | 0.916     
SAVOIAS Suprematism.csv        |     -0.657 |      0.616 | symmetry   | 0.598     
SAVOIAS Interior Design.csv    |     -0.709 |      0.666 | symmetry   | 0.571     
IC960

### Correlation Between Features

In [19]:
import pandas as pd
import numpy as np
from scipy import stats
import os

def analyze_feature_correlation(feature_files, feature_x='color_gradient', feature_y='canny_edge_density', feature_base="../features"):
    """
    Analyze direct correlation between two features across datasets.
    """
    print(f"\nAnalyzing correlation between {feature_x} and {feature_y}")
    print("\n{:<30} | {:>12} | {:>12} | {:>10}".format("Dataset", "Correlation", "P-value", "N"))
    print("-" * 70)
    
    correlations = []
    total_samples = 0
    weighted_sum = 0
    
    for file_path in feature_files:
        try:
            # Read CSV
            full_path = os.path.join(feature_base, file_path)
            df = pd.read_csv(full_path)
            
            # Calculate correlation
            correlation, p_value = stats.spearmanr(df[feature_x], df[feature_y], nan_policy='omit')
            n_samples = len(df)
            
            # Store results
            correlations.append({
                'dataset': file_path.split('/')[-1].replace('_features.csv', ''),
                'correlation': correlation,
                'p_value': p_value,
                'n_samples': n_samples
            })
            
            # Update weighted average calculation
            total_samples += n_samples
            weighted_sum += correlation * n_samples
            
            # Print result for this dataset
            print("{:<30} | {:>12.3f} | {:>12.3e} | {:>10}".format(
                correlations[-1]['dataset'],
                correlation,
                p_value,
                n_samples
            ))
            
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue
    
    # Calculate weighted average
    weighted_avg = weighted_sum / total_samples if total_samples > 0 else 0
    
    # Print summary statistics
    print("\nSummary Statistics:")
    print("-" * 30)
    print(f"Simple Average: {np.mean([c['correlation'] for c in correlations]):.3f}")
    print(f"Weighted Average: {weighted_avg:.3f}")
    print(f"Standard Deviation: {np.std([c['correlation'] for c in correlations]):.3f}")
    print(f"Total Samples: {total_samples:,}")
    
    return correlations

# List of files to process
feature_files = [
    "RSIVL.csv",
    "VISC.csv",
    "IC9600 Abstract.csv",
    "IC9600 Paintings.csv",
    "IC9600 Scenes.csv",
    "SAVOIAS Objects.csv",
    "SAVOIAS Art.csv",
    "SAVOIAS Scenes.csv",
    "SAVOIAS Suprematism.csv",
    "SAVOIAS Interior Design.csv",
    "IC9600 Advertisement.csv",
    "IC9600 Architecture.csv",
    "IC9600 Person.csv",
    "IC9600 Transport.csv",
    "IC9600 Objects.csv",
    "SVG.csv"
]


# Run the analysis
results = analyze_feature_correlation(feature_files, feature_x='M6', feature_y='MSG')


Analyzing correlation between M6 and MSG

Dataset                        |  Correlation |      P-value |          N
----------------------------------------------------------------------
RSIVL.csv                      |        0.899 |    1.932e-18 |         49
VISC.csv                       |        0.813 |   9.504e-190 |        800
IC9600 Abstract.csv            |        0.860 |    0.000e+00 |       1210
IC9600 Paintings.csv           |        0.738 |   5.385e-207 |       1200
IC9600 Scenes.csv              |        0.808 |   1.992e-281 |       1216
SAVOIAS Objects.csv            |        0.844 |    2.225e-55 |        200
SAVOIAS Art.csv                |        0.784 |    9.591e-89 |        420
SAVOIAS Scenes.csv             |        0.853 |    7.672e-58 |        200
SAVOIAS Suprematism.csv        |        0.850 |    4.391e-29 |        100
SAVOIAS Interior Design.csv    |        0.890 |    3.656e-35 |        100
IC9600 Advertisement.csv       |        0.865 |   2.014e-318 |       106